# Recuperación de pacientes del archivo de XiO
---------

# Procedimiento

- Configurar las herramientas necesarias
- Recuperar mediante FTP los datos archivados de XiO (herramientas externas)
- Subir a **gDrive** de Radiofísica los datos archivados
- Introducir los parámetros relevantes
- Correr los scripts de recuparación
- Descargar los archivos recuperados
- Limpiar en gDrive los archivos creados

## Configuración de herramientas

Instalación de `plastimatch` y `tree`

In [1]:
%%capture
#!sudo apt update

!sudo apt install plastimatch

!sudo apt install tree

Instalación e importación de los módulos `pydicom` y `pyplastimatch`

In [2]:
%%capture

# Instalación de módulos
%pip install pydicom
%pip install pyplastimatch

# Importación de módulos
# - Manipulación de archivos DICOM
import pydicom as dicom
# - Conversión del formato nativo de XiO a DICOM
import pyplastimatch as plast

Cargar los módulos necesrios

In [3]:
# - Acceso a los datos en gDrive
from google.colab import drive
# - Rutas y comprobaciones de existencia de ficheros 
from os import path
from pathlib import Path
from glob import glob
from sys import path as syspath
# - Utilidades de archivo entre gDrive y el sistema local de archivos
from google.colab import files

Montar el disco

In [4]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Crear uan referencia al directorio base de trabajo, crearlo e ir a él

In [5]:
%%capture
# Referenciar
baseworkdir = '/content/gdrive/MyDrive/XiO'

# Crear el directorio
!mkdir -p $baseworkdir

# Ir al directorio base
%cd $baseworkdir

Crear la carpeta `scripts` en el directorio base de trabajo en **gDrive** y descargar los scripts que complementan el funcionamiento de `plastimatch`

In [6]:
%%capture
# Descargar los scripts de corrección
if Path(baseworkdir + '/scripts').exists():
  !rm -rf $baseworkdir/scripts
!mkdir -p $baseworkdir/scripts
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/xioss
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/genindex
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/deltact
!wget --directory-prefix $baseworkdir/scripts https://raw.githubusercontent.com/csarux/XiOrecovery/main/scripts/dcmcoher.py


# Añadir al sistema la ruta de los scripts
if baseworkdir + '/scripts' not in syspath:
  syspath.insert(0, baseworkdir + '/scripts')

# Comprobar si el plan genérico con un único campo está y si no descargarlo
if not Path('RTPlan.dcm').exists():
  !wget https://raw.githubusercontent.com/csarux/XiOrecovery/main/DICOMSets/RTPlan.dcm

## Subir a **gDrive** de Radiofísica los datos archivados

Arrastrar sobre la carpeta XiO el archivo `.tgz` recuperado

Listar pacientes archivados presentes en el directorio base

In [8]:
!ls -l $baseworkdir/*.tgz

-rw------- 1 root root 68361913 May 25 09:32 /content/gdrive/MyDrive/XiO/592706_GOMEZ_FERNANDEZ_ROSA_MAR_Sep_23_2011.tgz


Definir el fichero que contiene el nombre del paciente archivado

## Introducir los parámetros relevantes

In [9]:
ArchivedPatient = '592706_GOMEZ_FERNANDEZ_ROSA_MAR_Sep_23_2011.tgz'  #@param {type: "string"}

Extaer los datos archivados

In [10]:
%%capture
!tar -xvzf $ArchivedPatient

Inferir el nýumero de historia a partir del fichero archivado

In [11]:
PatientID = ArchivedPatient.split('_')[0]
PatientID

'592706'

Recuperar los apellidos y nombre del paciente almacenados en el fichero `demographic` de XiO 

In [12]:
demographicfile = !find ./patient/$PatientID -name demographic -print 
with open(demographicfile[0]) as demogf:
  demographicdata = demogf.read()

ApellidosNombre = demographicdata.split('\n')[2]
ApellidosNombre

'GOMEZ FERNANDEZ,ROSA MAR'

Visualizar la estructura de carpetas creadas

In [13]:
!tree -d patient

patient
└── 592706
    ├── anatomy
    │   └── studyset
    │       └── CT1
    │           ├── DCMData
    │           └── surfaces
    ├── dvh
    └── plan
        ├── BOOST
        ├── MAMAFSCIZD
        ├── PLANSETUPBOOST
        ├── PLANSETUPMAMA
        └── SUMA

13 directories


Definir las variables con la identificación del estudio CT y el nombre del plan

In [14]:
CT = 'CT1' #@param {type: "string"} 
Plan ='SUMA' #@param {type: "string"} 

Definir las variables de la prescripción

In [15]:
DosisTotal = 60.0  #@param {type: "number"} 
NumeroSesiones = 30 #@param {type: "integer"} 
DosisSesion =2.0 #@param {type: "number"} 

## Correr los scripts de recuperación

**Estrategia**

- Crear el índice de las imágenes de CT si no existe
- Recuperar las estructuras a partir de los archivos de texto `.msh` 
- Transformar estructuras y dosis a formato DICOM
- Introducir un plan básico genérico y asociarle a la dosis recpuerada y al conjunto de estructuras
- Establecer una marco de referencia coherente entre las imágenes CT y el conjunto plan, dosis, estructuras

In [16]:
# Si no existe crear el índice de imágenes CT y las máscaras `.mha` de todas las estructuras
if not Path(baseworkdir + '/patient/' + PatientID + '/anatomy/studyset/' + CT + '/index.dat').exists():
  # Copiar el script para la creación del índice de cortes de CT
  !cp scripts/genindex patient/$PatientID/anatomy/studyset/$CT

  # Cambiar el directorio de trabajo
  %cd patient/$PatientID/anatomy/studyset/$CT

  # Adjudicar permisos de ejecución y crear el índice
  !sudo chmod +x genindex
  !./genindex

  # Crear la carpeta para los archivos máscara con las estructuras
  !mkdir -p ./ss

  # Copiar el script para la creación de las estructuras
  !cp $baseworkdir/scripts/xioss .

  # Correr el script de recuperación de las estructuras
  !sudo chmod +x xioss
  !./xioss --input $baseworkdir/patient/$PatientID --plan $Plan > xioss.log

  # Cambiar el directorio de trabajo al raíz de los pacientes
  %cd $baseworkdir/patient

  # Copiar el script para determinar el delta entre los marcos de referencia del estudio CT y el plan
  !cp $baseworkdir/scripts/deltact .

  # Adjudicar permisos de ejecución y calcular el delta
  !sudo chmod +x deltact
  !./deltact --input $PatientID --studyset $CT 
  

# Regresar al directorio base
%cd $baseworkdir

# Variables de pyplastimatch
verbose = True
path_to_logfile_xiodcm = './' + PatientID + '_xiodcm.log'

# Opciones de plastimatch convert
convert_args_rt ={'input' :  'patient/' + PatientID ,
                  'output-dicom' : 'xiodcm'} 

# Correr el script de conversión de los datos XiO en estudio DICOM                   
plast.convert(verbose=verbose, path_to_log_file=path_to_logfile_xiodcm, **convert_args_rt )

# Extracción del delta entre los marcos de referencia del estudio CT y el plan a partir del archivo CT#.infor
## Cambiar el directorio de trabajo al raíz de los pacientes
%cd $baseworkdir/patient

# Copiar el script para determinar el delta entre los marcos de referencia del estudio CT y el plan
!cp $baseworkdir/scripts/deltact .

## Asegurar que el archivo se actualiza
if Path('./deltaframes').exists():
  !rm -f ./deltaframes
## Adjudicar permisos de ejecución y calcular el delta
!sudo chmod +x deltact
!./deltact --input $PatientID --studyset $CT > deltact.log

## Regresar al directorio base
%cd $baseworkdir

# Incorporar el plan genérico 
!cp RTPlan.dcm xiodcm

# Importar las herramientas de corrección
from dcmcoher import correctImagePositionPatientInCTImages, correctRTPlan

# Asociar al plan las dosis y estructuras
correctRTPlan()

# Establecer el mismo marco de referencia en todo el estudio
correctImagePositionPatientInCTImages(patientID=PatientID, studyset=CT)

/content/gdrive/MyDrive/XiO/patient/592706/anatomy/studyset/CT1
mv: 'ss/patient.mha' and 'ss/patient.mha' are the same file
mv: 'ss/externo.mha' and 'ss/externo.mha' are the same file
mv: 'ss/medula.mha' and 'ss/medula.mha' are the same file
mv: 'ss/pulmon izdo.mha' and 'ss/pulmon izdo.mha' are the same file
mv: 'ss/pulmon drcho.mha' and 'ss/pulmon drcho.mha' are the same file
mv: 'ss/corazon.mha' and 'ss/corazon.mha' are the same file
mv: 'ss/linea media.mha' and 'ss/linea media.mha' are the same file
mv: 'ss/PTV MAMA IZQ.mha' and 'ss/PTV MAMA IZQ.mha' are the same file
mv: 'ss/FSC+AXILA.mha' and 'ss/FSC+AXILA.mha' are the same file
mv: 'ss/BOOST  11.mha' and 'ss/BOOST  11.mha' are the same file
Use of uninitialized value in string eq at ./xioss line 151.
Consistency lack in structure match between XiO contour file and plastimatch prefix output at ./xioss line 151.
/content/gdrive/MyDrive/XiO/patient
Reading CT1.info file
Reference frame delta x[mm], y[mm]: 0.488280,-213.512000
Writin

IndexError: ignored

## Descargar los archivos recuperados

In [ ]:
!zip -rq xiodcm.zip xiodcm
files.download('xiodcm.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Limpiar gDrive

In [ ]:
!rm -rf xiodcm patient xiodcm.zip *.log $ArchivedPatient

In [ ]:
!rm -rf xiodcm patient xiodcm.zip *.log



-------
## Logfiles 

Revisión de *logfiles*

Log del plan

In [ ]:
!cat $PatientID'_xiodcm.log'

Log de estructuras

In [ ]:
!cat patient/$PatientID/anatomy/studyset/$CT/xioss.log

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)

In [ ]:
file_list = gdrive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  if file1['title'] == 'XiO' and file1['mimeType']=='application/vnd.google-apps.folder': file1.Delete()

In [31]:
# Copiar el script para determinar el delta entre los marcos de referencia del estudio CT y el plan
!cp $baseworkdir/scripts/deltact .

## Asegurar que el archivo se actualiza
if Path('./deltaframes').exists():
  !rm -f ./deltaframes
## Adjudicar permisos de ejecución y calcular el delta
!sudo chmod +x deltact
!./deltact --input $PatientID --studyset $CT > deltact.log

## Regresar al directorio base
%cd $baseworkdir

# Incorporar el plan genérico 
!cp RTPlan.dcm xiodcm

# Importar las herramientas de corrección
from dcmcoher import correctImagePositionPatientInCTImages, correctRTPlan

# Asociar al plan las dosis y estructuras
correctRTPlan()

# Establecer el mismo marco de referencia en todo el estudio
correctImagePositionPatientInCTImages(patientID=PatientID, studyset=CT)

592706/anatomy/studyset/ doesn't exit at ./deltact line 29.
/content/gdrive/MyDrive/XiO


In [33]:
correctImagePositionPatientInCTImages(patientID=PatientID, studyset=CT)